# comercio_ext_auxiliares.tb_auxiliar_paises
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/PAIS_delta`
## 📌 Descrição do arquivo
Referência de **Países** com códigos ISO e nomes multilíngues (PT/EN/ES).
|Coluna|Descrição|
|---|---|
|`CO_PAIS`|Código interno do país|
|`CO_PAIS_ISON3`|Código ISO numérico (3)|
|`CO_PAIS_ISOA3`|Código ISO alfabético (3)|
|`NO_PAIS`|Nome em PT|
|`NO_PAIS_ING`|Nome em EN|
|`NO_PAIS_ESP`|Nome em ES|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable

jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:

bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/pais/"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_auxiliares/tb_auxiliar_paises/"
silverTable = "tb_auxiliar_paises"



In [0]:

silverSchema = T.StructType([
    T.StructField("CO_PAIS",        T.StringType(),  nullable=False),
    T.StructField("CO_PAIS_ISON3",  T.StringType(),  nullable=False),
    T.StructField("CO_PAIS_ISOA3",  T.StringType(),  nullable=False),
    T.StructField("NO_PAIS",        T.StringType(),  nullable=False),
    T.StructField("NO_PAIS_ING",    T.StringType(),  nullable=False),
    T.StructField("NO_PAIS_ESP",    T.StringType(),  nullable=False),
    T.StructField("TS_REF",         T.TimestampType(), nullable=False),
    T.StructField("NM_ORIGEM",      T.StringType(),  nullable=False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze_raw = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
df_normalized = (
    df_bronze_raw
    .withColumn("CO_PAIS",        F.upper(F.trim(F.col("CO_PAIS").cast(T.StringType()))))
    .withColumn("CO_PAIS_ISON3",  F.upper(F.trim(F.col("CO_PAIS_ISON3").cast(T.StringType()))))
    .withColumn("CO_PAIS_ISOA3",  F.upper(F.trim(F.col("CO_PAIS_ISOA3").cast(T.StringType()))))
    .withColumn("NO_PAIS",        F.col("NO_PAIS").cast(T.StringType()))
    .withColumn("NO_PAIS_ING",    F.col("NO_PAIS_ING").cast(T.StringType()))
    .withColumn("NO_PAIS_ESP",    F.col("NO_PAIS_ESP").cast(T.StringType()))
)

In [0]:
df_with_defaults = (
    df_normalized
    .withColumn("TS_REF",    F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/balancacomercial/PAIS_delta"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_with_defaults.filter(F.col("CO_PAIS").isNotNull())


In [0]:
df_dedup = df_valid.dropDuplicates(["CO_PAIS"])

In [0]:

df_silver = df_dedup.select(
    "CO_PAIS",
    "CO_PAIS_ISON3",
    "CO_PAIS_ISOA3",
    "NO_PAIS",
    "NO_PAIS_ING",
    "NO_PAIS_ESP",
    "TS_REF",
    "NM_ORIGEM"
)


##Carga
> #### **overwrite**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_auxiliares.tb_auxiliar_paises")

merge_condition = """
  t.CO_PAIS   = s.CO_PAIS
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
      "CO_PAIS_ISON3": "s.CO_PAIS_ISON3",
      "CO_PAIS_ISOA3": "s.CO_PAIS_ISOA3",
      "NO_PAIS":       "s.NO_PAIS",
      "NO_PAIS_ING":   "s.NO_PAIS_ING",
      "NO_PAIS_ESP":   "s.NO_PAIS_ESP",
      "TS_REF":        "s.TS_REF",
      "NM_ORIGEM":     "s.NM_ORIGEM"
    })
    .whenNotMatchedInsertAll()
    .execute()
)

In [0]:
#display(spark.sql("select * from silver_comercio_ext_auxiliares.tb_auxiliar_paises"))